# Setup

## Import packages

In [ ]:
import pathlib
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from scipy.signal import convolve
from scipy.ndimage import gaussian_filter
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
from numpy.fft import fftn, fftshift
from tqdm.auto import tqdm
import os

import dask
import dask.array as da
from dask.diagnostics import ProgressBar
from dask_image.ndfourier import fourier_gaussian, _utils
import flox
import flox.xarray
from flox.xarray import xarray_reduce
import xrft

%matplotlib widget

## Setup dask client
Optional, without setting up anything, the arrays will be chunked but may not parallelize as expected

To set up with JupyterLab dask extension (if installed):
- Click on dask tab on the left side, just under the kernel tab and above the table of contents tab
- Start new cluster at bottom, then click the '<>' to insert a cell into the notebook with code to intialize the client
- Click Launch dashboard in output from inserted cell to see standard dashboard windows on right side
- In the dask tab, launch any other progress windows of intereste (graph is nice)

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:42979")
client

## Import local files

In [ ]:
from ptable_dict import ptable, atomic_masses
from utilities import write_xyz, load_xyz, rotation_matrix, gaussian_kernel
from meshgrids import generate_density_grid, convert_grid_qspace, plot_3D_grid, generate_electron_grid_npys_fixed, flexible_density_grid_npys_along_x
from detector import make_detector, rotate_about_normal, rotate_about_horizontal, rotate_about_vertical, intersect_detector

# Generate and plot real-space voxel map for xyz file

## Fixed voxel segmentations along x

In [ ]:
# Define base path
basePath = pathlib.Path('/nsls2/users/alevin/repos/giwaxs_forward_sim')
xyzPath = basePath.joinpath('test_xyz_files/graphite_medium.xyz')
npySavePath = pathlib.Path('/nsls2/users/alevin/misc_data/density_grid_segments/',
                           'graphite_medium_v6')
npySavePath.mkdir(exist_ok=True)

sigma = 0.1
voxel_size = 0.045
min_ax_size = 512
segments = 8  # segments along x, also for chunks, make reasonable for ram
num_chunks = 8 # chunks to reload 

shape_info = generate_electron_grid_npys_fixed(xyzPath, 
                                               voxel_size, 
                                               segments,
                                               npySavePath,
                                               sigma,
                                               min_ax_size=min_ax_size)
x_axis, y_axis, z_axis, grid_vox_x, grid_vox_y, grid_vox_z = shape_info

In [ ]:
client.restart()

In [ ]:
# Below loads the numpy array stacks into a dask array
# I've so far been unable to fit this all into the separate python script without 
# running into strange moduleimport errors... but this should work!
def load_array_from_npy_stack(npy_paths):
    arrs = []
    for npy_path in npy_paths:
        arr = np.load(npy_path)
        arrs.append(arr)

    return np.concatenate(arrs, axis=1)   

npy_paths = sorted(npySavePath.glob('*.npy'), key=lambda x: int(x.name.split('_')[3].split('-')[-1]))
density_grid = dask.delayed(load_array_from_npy_stack)(npy_paths)
density_grid = dask.array.from_delayed(density_grid, shape=(grid_vox_y, grid_vox_x, grid_vox_z), dtype=float)
density_grid = density_grid.rechunk((grid_vox_y, int(grid_vox_x/num_chunks), grid_vox_z))

density_grid = density_grid.persist()

# Put loaded dask array into xarray data_array
dens_grid_DA = xr.DataArray(data=density_grid,
                            dims=['y', 'x', 'z'],
                            coords={'y':y_axis,
                                    'x':x_axis,
                                    'z':z_axis})
dens_grid_DA

In [ ]:
# Compute sum along 'z' and plot result below
zsum_DA = dens_grid_DA.sum('z').compute()

In [ ]:
smeared_zsum_DA = xr.ones_like(zsum_DA)
smeared_zsum_DA.data = gaussian_filter(zsum_DA.data, 5)

# Plot
plt.close('all')
ax = smeared_zsum_DA.plot.imshow()
ax.axes.set(xlabel='x [Å]', ylabel='y [Å]')
plt.show()

## Flexible & smeared voxel segmentations along x
- "works", but not a recommended workflow. Better to applying gaussian to fft result later

In [ ]:
# Define base path
basePath = pathlib.Path('/nsls2/users/alevin/repos/giwaxs_forward_sim')
xyzPath = basePath.joinpath('test_xyz_files/graphite_medium.xyz')
npySavePath = pathlib.Path('/nsls2/users/alevin/misc_data/density_grid_segments/',
                           'graphite_medium_testing_v7')
npySavePath.mkdir(exist_ok=True)

sigma = 0.1
voxel_size = 0.045 / 2
min_ax_size = 512 * 2
segment_vox_size = 64  # if segments type is 'flexible', target number for grid_vox_x
num_chunks = 16  # number of chunks to reload dask array in as

shape_info = flexible_density_grid_npys_along_x(xyzPath, 
                                                npySavePath,
                                                voxel_size, 
                                                sigma,
                                                segment_vox_size,
                                                min_ax_size=min_ax_size)

x_axis, y_axis, z_axis, grid_vox_x, grid_vox_y, grid_vox_z = shape_info

# Below loads the numpy array stacks into a dask array
# I've so far been unable to fit this all into the separate python script without 
# running into strange moduleimport errors... but this should work!
def load_array_from_npy_stack(npy_paths):
    """"""
    arrs = []
    for npy_path in npy_paths:
        arr = np.load(npy_path)
        arrs.append(arr)

    return np.concatenate(arrs, axis=1)   

npy_paths = sorted(npySavePath.glob('*.npy'))
density_grid = dask.delayed(load_array_from_npy_stack)(npy_paths)
density_grid = dask.array.from_delayed(density_grid, shape=(grid_vox_y, grid_vox_x, grid_vox_z), dtype=float)
density_grid = density_grid.rechunk((grid_vox_y, int(grid_vox_x/num_chunks), grid_vox_z))

density_grid = density_grid.persist()

# Put loaded dask array into xarray data_array
dens_grid_DA = xr.DataArray(data=density_grid,
                            dims=['y', 'x', 'z'],
                            coords={'y':y_axis,
                                    'x':x_axis,
                                    'z':z_axis})
dens_grid_DA

In [ ]:
# # Compute sum along 'z', apply gaussian filter, plot below
# zsum_DA = dens_grid_DA.sum('z').compute()

In [ ]:
# # Quick plot summing along z
# plt.close('all')
# cmin,cmax = zsum_DA.quantile([0.01, 0.999])
# zsum_DA.plot.imshow(cmap=plt.cm.viridis, norm=plt.Normalize(cmin,cmax))
# plt.show()

## Work in progress: code to down sample 3D output

In [ ]:
lazy_binned_DA = dens_grid_DA.groupby_bins('x', 128).mean().groupby_bins('y',128).mean().groupby_bins('z',128).mean()

In [ ]:
lazy_binned_DA.data.visualize()

In [ ]:
binned_DA = lazy_binned_DA.persist()

display(binned_DA)

In [ ]:
binned_DA = binned_DA.assign_coords({
            'x': ('x_bins', np.array([interval.mid for interval in binned_DA.x_bins.data])),
            'y': ('y_bins', np.array([interval.mid for interval in binned_DA.y_bins.data])),
            'z': ('z_bins', np.array([interval.mid for interval in binned_DA.z_bins.data]))
                   }).swap_dims({'x_bins':'x', 'y_bins':'y', 'z_bins':'z'})
binned_DA

In [ ]:
plt.close('all')
threshold = 99.9
num_levels = 10
cmap = 'plasma'
fig, ax = plot_3D_grid(density_grid.compute(), x_axis, y_axis, z_axis, cmap, threshold, num_levels, log=True)
# fig, ax = plot_3D_grid(binned_DA.data.compute(), binned_DA.x.data, binned_DA.y.data, binned_DA.z.data, cmap, threshold, num_levels, log=True)

plt.show()

# Generate and plot reciprocal space voxel map for xyz file

## Forward FFT with gaussian

In [ ]:
# Define fft parallelized function with xrft + dask + xarray!
def xrft_fft(DA, num_chunks):
    fft_yz = xrft.fft(DA, dim=['y','z'], shift=True)  # take dft in y & z direction
    fft_yz_rechunked = fft_yz.chunk({'freq_y':int(len(DA.y))/num_chunks,'x':int(len(DA.x))})  # rechunk along y direction 
    fft_all = xrft.fft(fft_yz_rechunked, dim=['x'], shift=True)  # take dft in x direction
    # fft_smeared = fourier_gaussian(fft_all, sigma)
    # iq_DA = np.abs(fft_all)**2
    
    return fft_all

# Take fft (lazily)
fft_DA = xrft_fft(dens_grid_DA, num_chunks)  # num chunks defined earlier when loading dens_grid
fft_DA = fft_DA.assign_coords({
            'qx': ('freq_x', fft_DA.freq_x.data * 2 * np.pi),
            'qy': ('freq_y', fft_DA.freq_y.data * 2 * np.pi),
            'qz': ('freq_z', fft_DA.freq_z.data * 2 * np.pi)
                   }).swap_dims({'freq_x':'qx', 'freq_y':'qy', 'freq_z':'qz'})


# # Optionally, apply gaussian (multiply by gaussian (analytically fourier transformed sigma value))
# def fft_gaussian_kernel(DA, sigma):
#     qx, qy, qz = da.meshgrid(DA.qy.data, DA.qx.data, DA.qz.data,)
#     g_fft = np.exp(2 * np.pi * (qx**2 + qy**2 + qz**2) * (sigma**2))  # 2π already applied in qx,qy,qz
    
#     return g_fft

# # Multiply gaussian (lazily)
# fft_DA = fft_DA * fft_gaussian_kernel(fft_DA, 0.05)

# Run computation graph with dask and persist result into memroy
fft_DA = fft_DA.persist()
fft_DA

In [ ]:
# Optionally, apply gaussian (multiply by gaussian (analytically fourier transformed sigma value))
def fft_gaussian_kernel(DA, sigma):
    qx, qy, qz = da.meshgrid(DA.qx.data, DA.qy.data, DA.qz.data,)
    g_fft = np.exp((qx**2 + qy**2 + qz**2) * (sigma**2))  # 2π already applied in qx,qy,qz
    
    return g_fft

# Multiply gaussian (lazily)
gauss_kern = fft_gaussian_kernel(fft_DA, 0.05)
gauss_kern

## Visualize 3D reciprocal space

In [ ]:
# fft_DA.data.visualize()

In [ ]:
extent = 10

iq_DA = np.abs(fft_DA)**2

# Plot
plt.close('all')
iq_DA_sum = iq_DA.sum('qz').compute()
sel_DA = iq_DA_sum.sel(qy=slice(-extent,extent),qx=slice(-extent,extent))
# sel_DA = iq_DA_sum.sel(qx=slice(None,None),qy=slice(None,None))
cmin,cmax = sel_DA.quantile([0.1,0.999])
ax = sel_DA.plot.imshow(norm=plt.Normalize(cmin,cmax))
plt.show()

In [ ]:
# 3D plotter, curently needs to compute & hold whole array in memory!

extent = 7.5

plt.close('all')
threshold = 98
num_levels = 10
cmap = 'plasma'

iq_DA = np.abs(fft_DA)**2
sel_DA = iq_DA.sel(qx=slice(-extent,extent), qy=slice(-extent,extent), qz=slice(-extent,extent))

fig, ax = plot_3D_grid(sel_DA.data.compute(), sel_DA.qx.data, sel_DA.qy.data, sel_DA.qz.data, cmap, threshold, num_levels)

plt.show()

## Optional Inverse FFT check  

In [ ]:
def xrft_ifft(DA, num_chunks):
    ifft_xz = xrft.ifft(DA, dim=['freq_x','freq_z'], shift=True, lag=(None,None))  # take dft in y & z direction
    ifft_xz_rechunked = ifft_xz.chunk({'x':int(len(DA.freq_x))/num_chunks,'freq_y':int(len(DA.freq_y))})  # rechunk along y direction 
    ifft_all = xrft.ifft(ifft_xz_rechunked, dim=['freq_y'], shift=True, lag=(None,))  # take dft in x direction
    
    return ifft_all

# ifft_all = xrft_ifft(fft_DA.swap_dims({'qx':'freq_x', 'qy':'freq_y', 'qz':'freq_z'}), num_chunks)
ifft_all = xrft_ifft((fft_DA*gauss_kern).swap_dims({'qx':'freq_x', 'qy':'freq_y', 'qz':'freq_z'}), num_chunks)
ifft_all = ifft_all.persist()
ifft_all

In [ ]:
# Compute sum along 'z' and plot result below
ifft_mag = np.abs(ifft_all)**2
zsum_DA = ifft_mag.sum('z').compute()

In [ ]:
## With FFT gauss sigma = 5!!
# smeared_zsum_DA = xr.ones_like(zsum_DA)
# smeared_zsum_DA.data = gaussian_filter(zsum_DA.data, 2)

# Plot
plt.close('all')
ax = zsum_DA.plot.imshow()
# ax = smeared_zsum_DA.plot.imshow()
ax.axes.set(xlabel='x [Å]', ylabel='y [Å]')
plt.show()

In [ ]:
## With FFT gauss sigma = 5!!
smeared_zsum_DA = xr.ones_like(zsum_DA)
smeared_zsum_DA.data = gaussian_filter(zsum_DA.data, 2)

# Plot
plt.close('all')
# ax = zsum_DA.plot.imshow()
ax = smeared_zsum_DA.plot.imshow()
ax.axes.set(xlabel='x [Å]', ylabel='y [Å]')
plt.show()

# find q-resolutions
### The frequency resolution (qbin size) is given by sampling rate (1/voxel_size) over box size (size of molecule)

In [ ]:
x_vals = qx
y_vals = qy
z_vals = qz
qx_res = x_vals[1]-x_vals[0]
qy_res = y_vals[1]-y_vals[0]
qz_res = z_vals[1]-z_vals[0]
print(f'Resolutions are [qx={qx_res:.4f}, qy={qy_res:.4f}, qz={qz_res:.4f}]')

In [ ]:
x_vals = iq_DA.qx.data
y_vals = iq_DA.qy.data
z_vals = iq_DA.qz.data
qx_res = x_vals[1]-x_vals[0]
qy_res = y_vals[1]-y_vals[0]
qz_res = z_vals[1]-z_vals[0]
print(f'Resolutions are [qx={qx_res:.4f}, qy={qy_res:.4f}, qz={qz_res:.4f}]')

# Set up Detector

In [ ]:
det_pixels = (200,200) #horizontal, vertical
det_qs = (8,8) #horizontal, vertical (these are absolute maximums. detector centered at 0)
det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])

psi = 0 #rotation in degrees of detector about detector normal axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_normal(det_x_grid, det_y_grid, det_z_grid, psi)
phi = 0 #rotation in degrees of detector about detector vertical axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_vertical(det_x_grid, det_y_grid, det_z_grid, phi)
theta = 0 #rotation in degrees of detector about detector horizontal axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_horizontal(det_x_grid, det_y_grid, det_z_grid, theta)

# plot single detector

In [ ]:
det_ints = intersect_detector(iq, qx, qy, qz, det_x_grid, det_y_grid, det_z_grid, det_h, det_v)

# plot
fig, ax1 = subplots()
ax1.imshow(det_ints,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(det_ints, 10), vmax=np.percentile(det_ints, 99)),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
ax1.set_xlabel('q horizontal')
ax1.set_ylabel('q vertical')

In [ ]:
det_ints = intersect_detector(iq_DA.data, iq_DA.qx.data*2*np.pi, iq_DA.qy.data*2*np.pi, iq_DA.qz.data*2*np.pi, det_x_grid, det_y_grid, det_z_grid, det_h, det_v)

# plot
fig, ax1 = subplots()
ax1.imshow(det_ints,
           norm=matplotlib.colors.Normalize(vmin=np.percentile(det_ints, 10), vmax=np.percentile(det_ints, 99)),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
ax1.set_xlabel('q horizontal')
ax1.set_ylabel('q vertical')

# Generate and sum multiple plots across selected angles

In [ ]:
def generate_detector_ints(det_pixels, det_qs, psi, phi, theta):
    det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])
    
    # psi = 0 #rotation in degrees of detector about detector normal axis
    det_x_grid, det_y_grid, det_z_grid = rotate_about_normal(det_x_grid, det_y_grid, det_z_grid, psi)
    # phi = 0 #rotation in degrees of detector about detector vertical axis
    det_x_grid, det_y_grid, det_z_grid = rotate_about_vertical(det_x_grid, det_y_grid, det_z_grid, phi)
    # theta = 0 #rotation in degrees of detector about detector horizontal axis
    det_x_grid, det_y_grid, det_z_grid = rotate_about_horizontal(det_x_grid, det_y_grid, det_z_grid, theta)
    det_ints = intersect_detector(iq, qx, qy, qz, det_x_grid, det_y_grid, det_z_grid, det_h, det_v)

    return det_ints

In [ ]:
#setup detector
det_pixels = (150,150) #horizontal, vertical
det_qs = (6.5,6.5) #horizontal, vertical (these are absolute maximums. detector centered at 0)
psi = 0 #rotation in degrees of detector about detector normal axis
phis = np.linspace(0,180,num=60) #rotation in degrees of detector about detector vertical axis
theta = 0 #rotation in degrees of detector about detector horizontal axis

det_ints = []
det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])
for i, phi in enumerate(phis):
    det_int = generate_detector_ints(det_pixels, det_qs, psi, phi, theta)
    if i == 0:
        det_sum = det_int
    else:
        det_sum +=det_int
    det_ints.append(det_int)

In [ ]:
%matplotlib widget
fig, ax1 = subplots()
cax = ax1.imshow(det_sum,
           norm=matplotlib.colors.LogNorm(vmin=np.percentile(det_sum, 30), vmax=np.percentile(det_sum, 99)),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
ax1.set_xlabel('q horizontal')
ax1.set_ylabel('q vertical')
ax1.set_xlim(left=0)
ax1.set_ylim(bottom=0)
cbar = fig.colorbar(cax, ax=ax1)

# Visualize each individual detector across angles

In [ ]:
%matplotlib inline

In [ ]:
for i in range(len(det_ints[:,0,0])):
    det_int = det_ints[i,:,:]
    fig, ax1 = subplots()
    cax = ax1.imshow(det_int,
           norm=matplotlib.colors.LogNorm(vmin=np.percentile(det_int, 10), vmax=np.percentile(det_int, 99)),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
    ax1.set_xlabel('q horizontal')
    ax1.set_ylabel('q vertical')
    ax1.set_xlim(0, 3)
    ax1.set_ylim(0, 3)
    cbar = fig.colorbar(cax, ax=ax1)
    ax1.set_title(f'Phi = {i*3} degrees')
    plt.show()
    plt.close('all')
    